In [1]:
import math as m
import numpy as np
import random as r
import matplotlib.pyplot as plt
import sys

In [2]:
import torch
from torch import nn
from torch import optim
from torch.optim.lr_scheduler import MultiStepLR

In [3]:
from nflows.flows.base import Flow
from nflows.distributions.uniform import BoxUniform
from nflows.transforms.base import CompositeTransform
from nflows.transforms.autoregressive import MaskedPiecewiseRationalQuadraticAutoregressiveTransform
from nflows.transforms.autoregressive import MaskedPiecewiseQuadraticAutoregressiveTransform
from nflows.transforms.permutations import ReversePermutation
from nflows.transforms.permutations import RandomPermutation
from nflows.transforms.splines.rational_quadratic import rational_quadratic_spline
from torch.utils.tensorboard import SummaryWriter

In [4]:
import subprocess
import time
import os
from copy import deepcopy
import math as m
import gc

## Tensorboard writer for loss logging

In [5]:
writer = SummaryWriter()

## GPU/CPU selection

In [6]:
device = torch.device("cuda:0")
#device = torch.device("cpu")

## Hyperparameters

In [7]:
n_RQS_knots = 10   # Number of knots in RQS transform
n_made_layers = 1  # Number of hidden layers in every made network
n_made_units = 100 # Number of units in every layer of the made network
n_flow_layers = 6  # Number of layers in the flow

batch_size = 1024
n_epochs = 800
adam_lr = 0.001     # Learning rate for the ADAM optimizer (default: 0.001)

n_train = int(1e6)  # Number of training events
n_test = int(1e5)   # Number of testing events
n_sample = int(1e6) # Number of samples for ess evaluation

## Load the training data

In [8]:
samples = np.genfromtxt("data/unweighted_samples.csv", delimiter=',')
if (n_train + n_test > samples.shape[0]):
    raise Exception("Not enough training data")

## Split to a train and test set

In [9]:
train_samples = torch.tensor(samples[:n_train], dtype=torch.float32, device=device)
test_samples = torch.tensor(samples[n_train:n_train+n_test], dtype=torch.float32, device=device)

del samples
gc.collect()

59

## Set up the flow

In [10]:
event_dim = train_samples.shape[1]
base_dist = BoxUniform(torch.zeros(event_dim), torch.ones(event_dim))

transforms = []
for _ in range(n_flow_layers):
    transforms.append(RandomPermutation(features=event_dim))
    transforms.append(MaskedPiecewiseRationalQuadraticAutoregressiveTransform(
        features=event_dim, 
        hidden_features=n_made_units,
        num_bins=n_RQS_knots,
        num_blocks=n_made_layers-1,
        tails="constrained",
        use_residual_blocks=False
    ))
transform = CompositeTransform(transforms)

flow = Flow(transform, base_dist).to(device)
optimizer = optim.Adam(flow.parameters(), lr=adam_lr)

scheduler = MultiStepLR(optimizer, milestones=[350, 425, 500, 575, 650, 725, 800], gamma=0.5)

## Training

In [11]:
data_size = train_samples.shape[0]
n_batches = m.ceil(data_size/batch_size)

data_size_validation = test_samples.shape[0]
n_batches_validate = m.ceil(data_size_validation/batch_size)

best_validation_loss = np.inf
best_ess = 0

for epoch in range(n_epochs):
    
    permutation = torch.randperm(data_size, device=device)    

    # Loop over batches
    cum_loss = 0
    for batch in range(n_batches):
        # Set up the batch
        batch_begin = batch*batch_size
        batch_end   = min( (batch+1)*batch_size, data_size-1 )
        indices = permutation[batch_begin:batch_end]
        samples_batch = train_samples[indices]
        
        # Take a step
        optimizer.zero_grad()
        loss = -(flow.log_prob(inputs=samples_batch)).mean()
        loss.backward()
        optimizer.step()

        # Compute cumulative loss
        cum_loss = (cum_loss*batch + loss.item())/(batch+1)

        if batch%25 == 0:
            print("epoch = ", epoch, "batch = ", batch, "/", n_batches, "loss = ", cum_loss)
    
    writer.add_scalar("Loss_train", cum_loss, epoch)
    scheduler.step()


    # ---------- Compute validation loss -----------
    validation_loss = 0
    for batch in range(n_batches_validate):
        batch_begin = batch*batch_size
        batch_end = min( (batch+1)*batch_size, data_size_validation-1 )
        samples_batch = test_samples[batch_begin:batch_end]
    
        with torch.no_grad():
            validation_loss = (validation_loss*batch - (flow.log_prob(samples_batch)).mean())/(batch+1)

    print("Validation loss = ", validation_loss.item())
    writer.add_scalar("Loss_test", validation_loss.item(), epoch)

    if validation_loss < best_validation_loss:
        torch.save(flow, "flow_model_unweighted_best_validation.pt")
        best_validation_loss = validation_loss

        
torch.save(flow, "flow_model_unweighted_final.pt")

epoch =  0 batch =  0 / 977 loss =  -1.0884255170822144
epoch =  0 batch =  25 / 977 loss =  -2.8339859522306003
epoch =  0 batch =  50 / 977 loss =  -4.326266354205561
epoch =  0 batch =  75 / 977 loss =  -6.912869434607656
epoch =  0 batch =  100 / 977 loss =  -8.86032288145311
epoch =  0 batch =  125 / 977 loss =  -10.188018113847765
epoch =  0 batch =  150 / 977 loss =  -11.185664401149117
epoch =  0 batch =  175 / 977 loss =  -11.996207131580872
epoch =  0 batch =  200 / 977 loss =  -12.677316544660881
epoch =  0 batch =  225 / 977 loss =  -13.263651326694319
epoch =  0 batch =  250 / 977 loss =  -13.77916301294152
epoch =  0 batch =  275 / 977 loss =  -14.234175676884856
epoch =  0 batch =  300 / 977 loss =  -14.630335736512345
epoch =  0 batch =  325 / 977 loss =  -14.975419168823336
epoch =  0 batch =  350 / 977 loss =  -15.276764960710139
epoch =  0 batch =  375 / 977 loss =  -15.54023440974824
epoch =  0 batch =  400 / 977 loss =  -15.775382934960344
epoch =  0 batch =  425 /

epoch =  3 batch =  525 / 977 loss =  -20.186301075460346
epoch =  3 batch =  550 / 977 loss =  -20.186334661909548
epoch =  3 batch =  575 / 977 loss =  -20.18696177999179
epoch =  3 batch =  600 / 977 loss =  -20.187734058018336
epoch =  3 batch =  625 / 977 loss =  -20.188008902552788
epoch =  3 batch =  650 / 977 loss =  -20.187697570261673
epoch =  3 batch =  675 / 977 loss =  -20.187590006540507
epoch =  3 batch =  700 / 977 loss =  -20.18846712915091
epoch =  3 batch =  725 / 977 loss =  -20.189507342567115
epoch =  3 batch =  750 / 977 loss =  -20.191064120926306
epoch =  3 batch =  775 / 977 loss =  -20.19082364839375
epoch =  3 batch =  800 / 977 loss =  -20.192280296678085
epoch =  3 batch =  825 / 977 loss =  -20.1935676893368
epoch =  3 batch =  850 / 977 loss =  -20.194652978737945
epoch =  3 batch =  875 / 977 loss =  -20.19473026223377
epoch =  3 batch =  900 / 977 loss =  -20.19576243105261
epoch =  3 batch =  925 / 977 loss =  -20.19688218819141
epoch =  3 batch =  95